<a href="https://colab.research.google.com/github/PradhyumnaPrakash/Ocular-Disease-Classification-and-Clustering-/blob/main/Ocular_Disease_Project_Full_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Supervised Learning


**Dataset 1 - Preprocessing**

In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#To merge all data into one folder
import os
import shutil

# paths
train_dir = "/content/drive/My Drive/Ocular Disease Dataset 1/Balanced Training Data"
test_dir = "/content/drive/My Drive/Ocular Disease Dataset 1/Balanced Training Data"
output_dir = "/content/drive/My Drive/Ocular Disease Dataset 1/Balanced Training Data"

# make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# get list of class folders from training (same in testing)
classes = os.listdir(train_dir)

for cls in classes:
    print(f"Processing {cls}...")

    # create destination subfolder
    dest_folder = os.path.join(output_dir, cls)
    os.makedirs(dest_folder, exist_ok=True)

    # source folders
    train_folder = os.path.join(train_dir, cls)
    test_folder = os.path.join(test_dir, cls)

    # combine both sources
    for folder in [train_folder, test_folder]:
        if not os.path.exists(folder):
            continue
        for file in os.listdir(folder):
            src_path = os.path.join(folder, file)
            dest_path = os.path.join(dest_folder, file)

            # if duplicate filename, rename before moving
            if os.path.exists(dest_path):
                base, ext = os.path.splitext(file)
                i = 1
                while os.path.exists(os.path.join(dest_folder, f"{base}_{i}{ext}")):
                    i += 1
                dest_path = os.path.join(dest_folder, f"{base}_{i}{ext}")

            shutil.move(src_path, dest_path)

print("Finished merging into 'class-wise dataset'")


In [ ]:
#Find individual class sizes

import os

data_dir = "/content/drive/My Drive/Ocular Disease Dataset 1/Class-wise Data"

class_counts = {}
for cls in os.listdir(data_dir):
    cls_path = os.path.join(data_dir, cls)
    if os.path.isdir(cls_path):
        class_counts[cls] = len(os.listdir(cls_path))

print("Class counts:", class_counts)

max_count = max(class_counts.values())
print("Majority class size:", max_count)


In [ ]:
#Finds image number from image name
def number_parser(num_string):
  number = ''
  for i in num_string:
    if i!='_':
      number = number+i
    else:
      break
  return number

In [ ]:
#Subject-level split

import random
random.seed(90)
data = pd.read_csv("/content/drive/MyDrive/Ocular Disease Dataset 1/full_df.csv")
list_main = []
list_training = []
for i in range(6392):
  list_main.append([data['filename'][i], data['target'][i]])
random.shuffle(list_main)
for k in range(20):
  print(list_main[k])
print('          ')
print('          ')
for i in range(2237):
  a = list_main[i]
  a_name = a[0]
  a_num = number_parser(a_name)
  if "right" in a_name:
    b_name = a_num+"_left.jpg"
    for k, sublist in enumerate(list_main):
      if b_name in sublist:
        b = list_main[k]
        break
  list_training.append(a)
  list_training.append(b)
for e in range(20):
  print(list_training[e])

In [ ]:
#Store pairs in dictionary and split
import random
subject_map = {}
for i in range(len(data)):
    fname = data['filename'][i]
    target = data['target'][i]
    subj_id = fname.split("_")[0]   # "1485"
    subject_map.setdefault(subj_id, []).append([fname, target])
subjects = list(subject_map.values())
random.seed(90)
random.shuffle(subjects)

# Split 70/30
n_train = int(len(subjects) * 0.7)
train_subjects = subjects[:n_train]
test_subjects = subjects[n_train:]

In [ ]:
#Flatten into training, testing list
list_training = [sample for subj in train_subjects for sample in subj]
list_testing  = [sample for subj in test_subjects for sample in subj]

print("Training images:", len(list_training))
print("Testing images:", len(list_testing))


In [ ]:
#Move each image into training folder
import os
import shutil

src_folder = "/content/drive/My Drive/Ocular Disease Dataset 1/preprocessed_images"
for value in list_training:
  name = value[0]
  disease = ''
  if value[1][1] == '1':
    disease = "Normal"
  elif value[1][4] == '1':
    disease = "Diabetes"
  elif value[1][7] == '1':
    disease = "Glaucoma"
  elif value[1][10] == '1':
    disease = "Cataract"
  elif value[1][13] == '1':
    disease = "ARMD"
  elif value[1][16] == '1':
    disease = "Hypertension"
  elif value[1][19] == '1':
    disease = "Myopia"
  elif value[1][22] == '1':
    disease = "Other"
  dst_folder = f"/content/drive/My Drive/Ocular Disease Dataset 1/Training Data/{disease}"
  src_path = os.path.join(src_folder, name)
  dst_path = os.path.join(dst_folder, name)
  shutil.copy(src_path, dst_path)

In [ ]:
#Moves each image into testing folder
import os
import shutil

src_folder = "/content/drive/My Drive/Ocular Disease Dataset 1/preprocessed_images"
for value in list_testing:
  name = value[0]
  disease = ''
  if value[1][1] == '1':
    disease = "Normal"
  elif value[1][4] == '1':
    disease = "Diabetes"
  elif value[1][7] == '1':
    disease = "Glaucoma"
  elif value[1][10] == '1':
    disease = "Cataract"
  elif value[1][13] == '1':
    disease = "ARMD"
  elif value[1][16] == '1':
    disease = "Hypertension"
  elif value[1][19] == '1':
    disease = "Myopia"
  elif value[1][22] == '1':
    disease = "Other"
  dst_folder = f"/content/drive/My Drive/Ocular Disease Dataset 1/Testing Data/{disease}"
  src_path = os.path.join(src_folder, name)
  dst_path = os.path.join(dst_folder, name)
  shutil.copy(src_path, dst_path)

In [ ]:
#Splits into 80% training, 20% validation
import os
import shutil
import random

train_base = "/content/drive/My Drive/Ocular Disease Dataset 1/Training Data"
val_base   = "/content/drive/My Drive/Ocular Disease Dataset 1/Validation Data"

# Make sure validation subfolders exist
classes = os.listdir(train_base)  # expects 8 subfolders already created
for cls in classes:
    os.makedirs(os.path.join(val_base, cls), exist_ok=True)
split_ratio = 0.2  # 20% validation
random.seed(90)    # reproducibility

for cls in classes:
    train_cls_folder = os.path.join(train_base, cls)
    val_cls_folder   = os.path.join(val_base, cls)

    images = os.listdir(train_cls_folder)

    random.shuffle(images)
    n_val = int(len(images) * split_ratio)
    val_images = images[:n_val]

    # move each selected image into validation
    for img in val_images:
        src = os.path.join(train_cls_folder, img)
        dst = os.path.join(val_cls_folder, img)
        shutil.move(src, dst)

    print(f"{cls}: moved {n_val} → validation, kept {len(images) - n_val} in training")


In [ ]:
#Oversampling and Data Augmentation for Training Dataset

import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

# Paths
src_base = "/content/drive/My Drive/Ocular Disease Dataset 1/Training Data"
out_base = "/content/drive/My Drive/Ocular Disease Dataset 1/Balanced_Training_Data"

classes = ["ARMD","Cataract","Diabetes","Glaucoma","Hypertension","Myopia","Normal","Other"]

# augmentation generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

# target count = Normal class count
target_count = 1617

for cls in classes:
    src_folder = os.path.join(src_base, cls)
    dst_folder = os.path.join(out_base, cls)
    os.makedirs(dst_folder, exist_ok=True)

    images = os.listdir(src_folder)
    count = len(images)

    print(f"{cls}: {count} images → target {target_count}")

    # Step 1. copy originals
    for img in images:
        src_path = os.path.join(src_folder, img)
        shutil.copy(src_path, os.path.join(dst_folder, img))

    # Step 2. oversample + augment minority classes
    i = 0
    while count < target_count:
        img_name = images[i % len(images)]  # cycle through originals
        img_path = os.path.join(src_folder, img_name)

        # load image
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        # generate one augmented image
        aug_iter = datagen.flow(x, batch_size=1)
        aug_img = next(aug_iter)[0].astype("uint8")

        # save with new name
        new_name = f"{os.path.splitext(img_name)[0]}_aug_{count}.jpg"
        array_to_img(aug_img).save(os.path.join(dst_folder, new_name))

        count += 1
        i += 1

    print(f"Balanced {cls}: {len(os.listdir(dst_folder))} images")

**Dataset 2 - Preprocessing**

In [ ]:
#Shuffle and separate each dataset into training and testing data

import os
import random
import shutil
random.seed(90)
source_1 = os.listdir('/content/drive/My Drive/Ocular Disease Dataset 2/dataset/normal')
source_2 = os.listdir('/content/drive/My Drive/Ocular Disease Dataset 2/dataset/diabetic_retinopathy')
source_3 = os.listdir('/content/drive/My Drive/Ocular Disease Dataset 2/dataset/glaucoma')
source_4 = os.listdir('/content/drive/My Drive/Ocular Disease Dataset 2/dataset/cataract')
random.shuffle(source_1)
random.shuffle(source_2)
random.shuffle(source_3)
random.shuffle(source_4)
training_data_1 = []
testing_data_1 = []
training_data_2 = []
testing_data_2 = []
training_data_3 = []
testing_data_3 = []
training_data_4 = []
testing_data_4 = []

for i in range(int(0.7*len(source_1))):
  training_data_1.append(source_1[i])
for j in range(int(0.7*len(source_1)), len(source_1)):
  testing_data_1.append(source_1[j])

for i in range(int(0.7*len(source_2))):
  training_data_2.append(source_2[i])
for j in range(int(0.7*len(source_2)), len(source_2)):
  testing_data_2.append(source_2[j])

for i in range(int(0.7*len(source_3))):
  training_data_3.append(source_3[i])
for j in range(int(0.7*len(source_3)), len(source_3)):
  testing_data_3.append(source_3[j])

for i in range(int(0.7*len(source_4))):
  training_data_4.append(source_4[i])
for j in range(int(0.7*len(source_4)), len(source_4)):
  testing_data_4.append(source_4[j])



In [ ]:
src_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/dataset/normal"
dst_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/Training Data/Normal"
for i in training_data_1:
  src_path = os.path.join(src_folder, i)
  dst_path = os.path.join(dst_folder, i)
  shutil.copy(src_path, dst_path)
print("Done With Normal Training")

src_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/dataset/normal"
dst_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/Testing Data/Normal"
for i in testing_data_1:
  src_path = os.path.join(src_folder, i)
  dst_path = os.path.join(dst_folder, i)
  shutil.copy(src_path, dst_path)
print("Done With Normal Testing")

src_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/dataset/diabetic_retinopathy"
dst_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/Training Data/Diabetes"
for i in training_data_2:
  src_path = os.path.join(src_folder, i)
  dst_path = os.path.join(dst_folder, i)
  shutil.copy(src_path, dst_path)
print("Done With Diabetes Training")

dst_folder_new = "/content/drive/My Drive/Ocular Disease Dataset 2/Testing Data/Diabetes"
for j in testing_data_2:
  src_path = os.path.join(src_folder, j)
  dst_path = os.path.join(dst_folder_new, j)
  shutil.copy(src_path, dst_path)
print("Done With Diabetes Testing")

src_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/dataset/glaucoma"
dst_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/Training Data/Glaucoma"
for i in training_data_3:
  src_path = os.path.join(src_folder, i)
  dst_path = os.path.join(dst_folder, i)
  shutil.copy(src_path, dst_path)
print("Done With Glaucoma Training")

dst_folder_new = "/content/drive/My Drive/Ocular Disease Dataset 2/Testing Data/Glaucoma"
for j in testing_data_3:
  src_path = os.path.join(src_folder, j)
  dst_path = os.path.join(dst_folder_new, j)
  shutil.copy(src_path, dst_path)
print("Done With Glaucoma Testing")

src_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/dataset/cataract"
dst_folder = "/content/drive/My Drive/Ocular Disease Dataset 2/Training Data/Cataract"
for i in training_data_4:
  src_path = os.path.join(src_folder, i)
  dst_path = os.path.join(dst_folder, i)
  shutil.copy(src_path, dst_path)
print("Done With Cataract Training")

dst_folder_new = "/content/drive/My Drive/Ocular Disease Dataset 2/Testing Data/Cataract"
for j in testing_data_4:
  src_path = os.path.join(src_folder, j)
  dst_path = os.path.join(dst_folder_new, j)
  shutil.copy(src_path, dst_path)
print("Done With Cataract Testing")

In [ ]:
#Create and separate validation data

import os
import shutil
import random

train_base = "/content/drive/My Drive/Ocular Disease Dataset 2/Training Data"
val_base   = "/content/drive/My Drive/Ocular Disease Dataset 2/Validation Data"

# Make sure validation subfolders exist
classes = os.listdir(train_base)  # expects 8 subfolders already created
for cls in classes:
    os.makedirs(os.path.join(val_base, cls), exist_ok=True)
split_ratio = 0.2  # 20% validation
random.seed(90)    # reproducibility

for cls in classes:
    train_cls_folder = os.path.join(train_base, cls)
    val_cls_folder   = os.path.join(val_base, cls)

    # get all images in this class
    images = os.listdir(train_cls_folder)

    # shuffle and split
    random.shuffle(images)
    n_val = int(len(images) * split_ratio)
    val_images = images[:n_val]

    # move each selected image into validation
    for img in val_images:
        src = os.path.join(train_cls_folder, img)
        dst = os.path.join(val_cls_folder, img)
        shutil.move(src, dst)

    print(f"{cls}: moved {n_val} → validation, kept {len(images) - n_val} in training")

In [ ]:
#Balance training data

import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

# Paths
train_base = "/content/drive/My Drive/Ocular Disease Dataset 2/Training Data"
balanced_train = "/content/drive/My Drive/Ocular Disease Dataset 2/Balanced Training Data"
os.makedirs(balanced_train, exist_ok=True)

# Augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Step 1: Count images per class
counts = {cls: len(os.listdir(os.path.join(train_base, cls))) for cls in os.listdir(train_base)}
max_count = max(counts.values())
print("Target count per class:", max_count)

# Step 2: Balance each class
for cls in os.listdir(train_base):
    src_folder = os.path.join(train_base, cls)
    dst_folder = os.path.join(balanced_train, cls)
    os.makedirs(dst_folder, exist_ok=True)

    images = os.listdir(src_folder)
    count = len(images)

    print(f"{cls}: {count} images → balancing to {max_count}")

    # Copy originals
    for img in images:
        shutil.copy(os.path.join(src_folder, img), os.path.join(dst_folder, img))

    # Oversample with augmentation
    i = 0
    while count < max_count:
        img_name = images[i % len(images)]  # cycle through originals
        img_path = os.path.join(src_folder, img_name)

        # Load image
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        # Generate augmented image
        aug_iter = datagen.flow(x, batch_size=1)
        aug_img = next(aug_iter)[0].astype("uint8")

        # Save augmented image
        new_name = f"{os.path.splitext(img_name)[0]}_aug_{count}.jpg"
        array_to_img(aug_img).save(os.path.join(dst_folder, new_name))

        count += 1
        i += 1

    print(f"{cls} balanced: {len(os.listdir(dst_folder))} images")

**Dataset 1 - Model Training and Evaluation**

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.applications import ResNet50, DenseNet121, VGG16

In [ ]:
# Load Data
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/Ocular Disease Dataset 1/Balanced_Training_Data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(512, 512))

val_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/Ocular Disease Dataset 1/Validation Data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(512, 512))

test_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/Ocular Disease Dataset 1/Testing Data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(512, 512))

In [ ]:
#Speed up training
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

ResNet50 - Model Training

In [ ]:
def build_model(base_model_name='ResNet50', input_shape=(512, 512, 3), num_classes=8):
    # Choose backbone
    if base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Choose from 'ResNet50', 'DenseNet121', or 'VGG16'")

    base_model.trainable = False  # Freeze base model

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
model = build_model(base_model_name='ResNet50', input_shape=(512, 512, 3), num_classes=8)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='pro_architecture_best_model_by_precision.weights.h5',
    monitor='val_precision',       # ← track validation precision
    mode='max',                    # because higher precision is better
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Summary
model.summary()

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=30)

#IMPORTANT: After training this model, skip the next two sections (DenseNet121 and VGG16 Model training) and directly begin with "Evaluations and Graphs" section

DenseNet121 - Model Training

In [ ]:
def build_model(base_model_name='DenseNet121', input_shape=(512, 512, 3), num_classes=8):
    # Choose backbone
    if base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Choose from 'ResNet50', 'DenseNet121', or 'VGG16'")

    base_model.trainable = False  # Freeze base model

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
model = build_model(base_model_name='DenseNet121', input_shape=(512, 512, 3), num_classes=8)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='pro_architecture_best_model_by_precision.weights.h5',
    monitor='val_precision',       # ← track validation precision
    mode='max',                    # because higher precision is better
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Summary
model.summary()

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=30)

VGG16 - Model Training

In [ ]:
def build_model(base_model_name='VGG16', input_shape=(512, 512, 3), num_classes=8):
    # Choose backbone
    if base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Choose from 'ResNet50', 'DenseNet121', or 'VGG16'")

    base_model.trainable = False  # Freeze base model

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
model = build_model(base_model_name='VGG16', input_shape=(512, 512, 3), num_classes=8)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='pro_architecture_best_model_by_precision.weights.h5',
    monitor='val_precision',       # ← track validation precision
    mode='max',                    # because higher precision is better
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Summary
model.summary()

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=30)

Evaluation and Graphs

In [ ]:
#Evaluation with Accuracy, Precision and Recall
loss, accuracy, precision, recall = model.evaluate(test_ds)
#print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

In [ ]:
# Plot training & validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')

In [ ]:
# Get true labels and predictions
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))      # Convert one-hot to class index
    y_pred.extend(np.argmax(preds, axis=1))                # Predicted class index
    print(labels.shape)
y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
# Classification report
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1','Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6', 'Class 7']))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1','Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6', 'Class 7'], yticklabels=['Class 0', 'Class 1','Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6', 'Class 7'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
#Class-wise accuracy, ROC and PR Curves, ROC-AUC and PR-AUC Values, CI

import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.metrics import (confusion_matrix, roc_auc_score, average_precision_score,
                             roc_curve, precision_recall_curve)
import matplotlib.pyplot as plt
from scipy.stats import norm

#Ensure test dataset has deterministic order
BATCH = 32
test_eval = test_ds.unbatch().batch(BATCH)  # preserves order
test_eval = test_eval.prefetch(tf.data.AUTOTUNE)

#Materialize arrays ONCE to guarantee alignment
X_list, Y_list = [], []
for x,y in test_eval:
    X_list.append(x.numpy())
    Y_list.append(y.numpy())
X = np.concatenate(X_list, axis=0)             # [N, H, W, C]
y_true_onehot = np.concatenate(Y_list, axis=0) # [N, 8]
y_true = y_true_onehot.argmax(axis=1)          # [N]

#Predict probabilities aligned with y_true
y_prob = model.predict(X, batch_size=BATCH, verbose=0)  # [N, 8]
y_pred = y_prob.argmax(axis=1)

# Sanity checks
assert y_prob.shape[0] == y_true.shape[0]
assert np.allclose(y_prob.sum(axis=1), 1, atol=1e-6)  # softmax

#3) Micro accuracy + Wilson CI
def wilson_ci(k, n, confidence=0.95):
    if n == 0: return (0.0, 0.0)
    z = norm.ppf(0.5 + confidence/2.0)
    phat = k/n
    denom = 1 + (z**2)/n
    center = (phat + (z**2)/(2*n)) / denom
    half = (z * ((phat*(1-phat)/n + (z**2)/(4*n**2))**0.5)) / denom
    return center - half, center + half

cm = confusion_matrix(y_true, y_pred, labels=np.arange(y_true_onehot.shape[1]))
diag = np.diag(cm)
total_correct = diag.sum()
total = cm.sum()

micro_acc = total_correct / total
lo_acc, hi_acc = wilson_ci(int(total_correct), int(total))
print(f"Micro accuracy = {micro_acc:.4f} (95% CI: {lo_acc:.4f}–{hi_acc:.4f})")

#Compare with Keras categorical accuracy to prove consistency
cat_acc = CategoricalAccuracy()
cat_acc.update_state(y_true_onehot, y_prob)
print(f"Keras CategoricalAccuracy = {cat_acc.result().numpy():.4f}")

#Per-class “accuracy” (row recall) + Wilson CI
row_sums = cm.sum(axis=1)
per_class_acc = diag / np.maximum(row_sums, 1)  # avoid /0

print("\nPer-class accuracy (i.e., recall/sensitivity):")
for i in range(len(per_class_acc)):
    k_i, n_i = int(diag[i]), int(row_sums[i])
    lo_i, hi_i = wilson_ci(k_i, n_i) if n_i > 0 else (0.0, 0.0)
    print(f"Class {i}: Acc={per_class_acc[i]:.4f} (95% CI: {lo_i:.4f}–{hi_i:.4f})")

#ROC-AUC & PR-AUC (per-class + micro & macro)
n_classes = y_true_onehot.shape[1]

#Per-class curves
plt.figure()
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_onehot[:, i], y_prob[:, i])
    plt.plot(fpr, tpr, label=f"Class {i}")
plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("Multiclass ROC"); plt.legend(); plt.show()

plt.figure()
for i in range(n_classes):
    prec, rec, _ = precision_recall_curve(y_true_onehot[:, i], y_prob[:, i])
    plt.plot(rec, prec, label=f"Class {i}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Multiclass PR"); plt.legend(); plt.show()

#AUC
roc_auc_per_class = [roc_auc_score(y_true_onehot[:, i], y_prob[:, i]) for i in range(n_classes)]
pr_auc_per_class  = [average_precision_score(y_true_onehot[:, i], y_prob[:, i]) for i in range(n_classes)]

roc_auc_micro = roc_auc_score(y_true_onehot, y_prob, average="micro", multi_class="ovr")
roc_auc_macro = roc_auc_score(y_true_onehot, y_prob, average="macro", multi_class="ovr")
pr_auc_micro  = average_precision_score(y_true_onehot, y_prob, average="micro")
pr_auc_macro  = average_precision_score(y_true_onehot, y_prob, average="macro")

print("\nROC-AUC per class:", [f"{v:.3f}" for v in roc_auc_per_class])
print("PR-AUC  per class:", [f"{v:.3f}" for v in pr_auc_per_class])
print(f"Micro ROC-AUC = {roc_auc_micro:.3f} | Macro ROC-AUC = {roc_auc_macro:.3f}")
print(f"Micro PR-AUC  = {pr_auc_micro:.3f}  | Macro PR-AUC  = {pr_auc_macro:.3f}")

#Bootstrap CIs for micro AUC
def bootstrap_ci(metric_fn, y_true_bin, y_score, n_boot=2000, confidence=0.95, seed=42):
    rng = np.random.default_rng(seed)
    n = y_true_bin.shape[0]
    stats = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        stats.append(metric_fn(y_true_bin[idx], y_score[idx]))
    lo = np.percentile(stats, (1-confidence)/2*100)
    hi = np.percentile(stats, (1+confidence)/2*100)
    return float(lo), float(hi)

roc_micro_ci = bootstrap_ci(lambda yt, yp: roc_auc_score(yt, yp, average="micro", multi_class="ovr"),
                            y_true_onehot, y_prob)
pr_micro_ci  = bootstrap_ci(lambda yt, yp: average_precision_score(yt, yp, average="micro"),
                            y_true_onehot, y_prob)

print(f"\nMicro ROC-AUC 95% CI: {roc_micro_ci[0]:.3f}–{roc_micro_ci[1]:.3f}")
print(f"Micro PR-AUC  95% CI: {pr_micro_ci[0]:.3f}–{pr_micro_ci[1]:.3f}")

**Dataset 2 - Model Training and Evaluation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.applications import ResNet50, DenseNet121, VGG16

In [ ]:
# Load Data
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/Ocular Disease Dataset 2/Balanced Training Data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(512, 512))

val_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/Ocular Disease Dataset 2/Validation Data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(512, 512))

test_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/Ocular Disease Dataset 2/Testing Data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(512, 512))

In [ ]:
#Speed up training
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

ResNet50 - Model Training

In [ ]:
def build_model(base_model_name='ResNet50', input_shape=(512, 512, 3), num_classes=4):
    # Choose backbone
    if base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Choose from 'ResNet50', 'DenseNet121', or 'VGG16'")

    base_model.trainable = False  # Freeze base model

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
model = build_model(base_model_name='ResNet50', input_shape=(512, 512, 3), num_classes=4)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='pro_architecture_best_model_by_precision.weights.h5',
    monitor='val_precision',       # ← track validation precision
    mode='max',                    # because higher precision is better
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Summary
model.summary()

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=30)

#IMPORTANT: After training this model, skip the next two sections (DenseNet121 and VGG16 Model training) and directly begin with "Evaluations and Graphs" section

DenseNet121 - Model Training

In [ ]:
def build_model(base_model_name='DenseNet121', input_shape=(512, 512, 3), num_classes=4):
    # Choose backbone
    if base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Choose from 'ResNet50', 'DenseNet121', or 'VGG16'")

    base_model.trainable = False  # Freeze base model

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
model = build_model(base_model_name='DenseNet121', input_shape=(512, 512, 3), num_classes=4)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='pro_architecture_best_model_by_precision.weights.h5',
    monitor='val_precision',       # ← track validation precision
    mode='max',                    # because higher precision is better
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Summary
model.summary()

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=30)

VGG16 - Model Training

In [ ]:
def build_model(base_model_name='VGG16', input_shape=(512, 512, 3), num_classes=4):
    # Choose backbone
    if base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Choose from 'ResNet50', 'DenseNet121', or 'VGG16'")

    base_model.trainable = False  # Freeze base model

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
model = build_model(base_model_name='VGG16', input_shape=(512, 512, 3), num_classes=4)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='pro_architecture_best_model_by_precision.weights.h5',
    monitor='val_precision',       # ← track validation precision
    mode='max',                    # because higher precision is better
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Summary
model.summary()

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=30)

Model Evaluation and Graphs

In [ ]:
loss, accuracy, precision, recall = model.evaluate(test_ds)
#print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test Precision: {precision:.4f}")

In [ ]:
# Plot training & validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')

In [ ]:
# Get true labels and predictions
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))      # Convert one-hot to class index
    y_pred.extend(np.argmax(preds, axis=1))                # Predicted class index
    print(labels.shape)
y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
# Classification report
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1','Class 2', 'Class 3']))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1','Class 2', 'Class 3'], yticklabels=['Class 0', 'Class 1','Class 2', 'Class 3'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
#Class-wise accuracy, ROC and PR Curves, ROC-AUC and PR-AUC Values, CI

import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.metrics import (confusion_matrix, roc_auc_score, average_precision_score,
                             roc_curve, precision_recall_curve)
import matplotlib.pyplot as plt
from scipy.stats import norm

#Ensure test dataset has deterministic order
BATCH = 32
test_eval = test_ds.unbatch().batch(BATCH)  # preserves order
test_eval = test_eval.prefetch(tf.data.AUTOTUNE)

#Materialize arrays ONCE to guarantee alignment
X_list, Y_list = [], []
for x,y in test_eval:
    X_list.append(x.numpy())
    Y_list.append(y.numpy())
X = np.concatenate(X_list, axis=0)             # [N, H, W, C]
y_true_onehot = np.concatenate(Y_list, axis=0) # [N, 8]
y_true = y_true_onehot.argmax(axis=1)          # [N]

#Predict probabilities aligned with y_true
y_prob = model.predict(X, batch_size=BATCH, verbose=0)  # [N, 8]
y_pred = y_prob.argmax(axis=1)

# Sanity checks
assert y_prob.shape[0] == y_true.shape[0]
assert np.allclose(y_prob.sum(axis=1), 1, atol=1e-6)  # softmax

#3) Micro accuracy + Wilson CI
def wilson_ci(k, n, confidence=0.95):
    if n == 0: return (0.0, 0.0)
    z = norm.ppf(0.5 + confidence/2.0)
    phat = k/n
    denom = 1 + (z**2)/n
    center = (phat + (z**2)/(2*n)) / denom
    half = (z * ((phat*(1-phat)/n + (z**2)/(4*n**2))**0.5)) / denom
    return center - half, center + half

cm = confusion_matrix(y_true, y_pred, labels=np.arange(y_true_onehot.shape[1]))
diag = np.diag(cm)
total_correct = diag.sum()
total = cm.sum()

micro_acc = total_correct / total
lo_acc, hi_acc = wilson_ci(int(total_correct), int(total))
print(f"Micro accuracy = {micro_acc:.4f} (95% CI: {lo_acc:.4f}–{hi_acc:.4f})")

#Compare with Keras categorical accuracy to prove consistency
cat_acc = CategoricalAccuracy()
cat_acc.update_state(y_true_onehot, y_prob)
print(f"Keras CategoricalAccuracy = {cat_acc.result().numpy():.4f}")

#Per-class “accuracy” (row recall) + Wilson CI
row_sums = cm.sum(axis=1)
per_class_acc = diag / np.maximum(row_sums, 1)  # avoid /0

print("\nPer-class accuracy (i.e., recall/sensitivity):")
for i in range(len(per_class_acc)):
    k_i, n_i = int(diag[i]), int(row_sums[i])
    lo_i, hi_i = wilson_ci(k_i, n_i) if n_i > 0 else (0.0, 0.0)
    print(f"Class {i}: Acc={per_class_acc[i]:.4f} (95% CI: {lo_i:.4f}–{hi_i:.4f})")

#ROC-AUC & PR-AUC (per-class + micro & macro)
n_classes = y_true_onehot.shape[1]

#Per-class curves
plt.figure()
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_onehot[:, i], y_prob[:, i])
    plt.plot(fpr, tpr, label=f"Class {i}")
plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("Multiclass ROC"); plt.legend(); plt.show()

plt.figure()
for i in range(n_classes):
    prec, rec, _ = precision_recall_curve(y_true_onehot[:, i], y_prob[:, i])
    plt.plot(rec, prec, label=f"Class {i}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Multiclass PR"); plt.legend(); plt.show()

#AUC
roc_auc_per_class = [roc_auc_score(y_true_onehot[:, i], y_prob[:, i]) for i in range(n_classes)]
pr_auc_per_class  = [average_precision_score(y_true_onehot[:, i], y_prob[:, i]) for i in range(n_classes)]

roc_auc_micro = roc_auc_score(y_true_onehot, y_prob, average="micro", multi_class="ovr")
roc_auc_macro = roc_auc_score(y_true_onehot, y_prob, average="macro", multi_class="ovr")
pr_auc_micro  = average_precision_score(y_true_onehot, y_prob, average="micro")
pr_auc_macro  = average_precision_score(y_true_onehot, y_prob, average="macro")

print("\nROC-AUC per class:", [f"{v:.3f}" for v in roc_auc_per_class])
print("PR-AUC  per class:", [f"{v:.3f}" for v in pr_auc_per_class])
print(f"Micro ROC-AUC = {roc_auc_micro:.3f} | Macro ROC-AUC = {roc_auc_macro:.3f}")
print(f"Micro PR-AUC  = {pr_auc_micro:.3f}  | Macro PR-AUC  = {pr_auc_macro:.3f}")

#Bootstrap CIs for micro AUC
def bootstrap_ci(metric_fn, y_true_bin, y_score, n_boot=2000, confidence=0.95, seed=42):
    rng = np.random.default_rng(seed)
    n = y_true_bin.shape[0]
    stats = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        stats.append(metric_fn(y_true_bin[idx], y_score[idx]))
    lo = np.percentile(stats, (1-confidence)/2*100)
    hi = np.percentile(stats, (1+confidence)/2*100)
    return float(lo), float(hi)

roc_micro_ci = bootstrap_ci(lambda yt, yp: roc_auc_score(yt, yp, average="micro", multi_class="ovr"),
                            y_true_onehot, y_prob)
pr_micro_ci  = bootstrap_ci(lambda yt, yp: average_precision_score(yt, yp, average="micro"),
                            y_true_onehot, y_prob)

print(f"\nMicro ROC-AUC 95% CI: {roc_micro_ci[0]:.3f}–{roc_micro_ci[1]:.3f}")
print(f"Micro PR-AUC  95% CI: {pr_micro_ci[0]:.3f}–{pr_micro_ci[1]:.3f}")

# Unsupervised Learning

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
from sklearn.manifold import TSNE
from scipy.stats import mode
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from matplotlib.patches import Patch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Dataset 1**

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/My Drive/Ocular Disease Dataset 1/Unsupervised Test",
    image_size=(224, 224),   # Resize all images
    batch_size=512,         # Get individual images
    shuffle=False            # Keep order for label tracking
)

In [ ]:
# Preprocessing using .map()
dataset = dataset.map(lambda x, y: (preprocess_input(x), y))

# Load pretrained model
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Extract features in batches
features = []
labels = []

for batch_images, batch_labels in dataset:
    batch_features = base_model.predict(batch_images)
    features.append(batch_features)
    labels.append(batch_labels.numpy())

features = np.concatenate(features)
labels = np.concatenate(labels)

In [ ]:
#Right now using K-Means, can try DBSCAN or Agglomerative Clustering
n_clusters = len(np.unique(labels))  # Number of actual classes
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
pred_clusters = kmeans.fit_predict(features)
# Now you have a feature matrix (features) for your images.

In [ ]:
ari = adjusted_rand_score(labels, pred_clusters)
print(f"Adjusted Rand Index: {ari:.4f}")

nmi = normalized_mutual_info_score(labels, pred_clusters)
print(f"NMI: {nmi:.4f}")

In [ ]:
X_2d = TSNE(n_components=2, random_state=42).fit_transform(features)

In [ ]:
# Ensure labels and clusters are numpy arrays
labels = np.array(labels)
pred_clusters = np.array(pred_clusters)

# Step 1: Define a consistent set of classes
all_classes = np.unique(np.concatenate([labels, pred_clusters]))
class_to_index = {cls: i for i, cls in enumerate(sorted(all_classes))}

# Step 2: Choose a colormap with enough colors
cmap = plt.get_cmap('tab10')  # Or use 'tab20' for more classes
num_colors = len(all_classes)
colors = [cmap(i % cmap.N) for i in range(num_colors)]  # wrap around if too many

# Step 3: Build class-to-color mapping
color_dict = {cls: colors[i] for cls, i in class_to_index.items()}

# Step 4: Map each label to a color using the dict
label_colors = [color_dict[l] for l in labels]
cluster_colors = [color_dict[c] for c in pred_clusters]

In [ ]:
def map_clusters_to_labels(true_labels, cluster_preds):
    labels = np.zeros_like(cluster_preds)
    for cluster_id in np.unique(cluster_preds):
        mask = cluster_preds == cluster_id
        labels[mask] = mode(true_labels[mask])[0]
    return labels

aligned_preds = map_clusters_to_labels(labels, pred_clusters)

In [ ]:
# Create color map based on unique true labels
unique_labels = np.unique(labels)
cmap = plt.get_cmap('tab10')
color_list = [cmap(i) for i in range(len(unique_labels))]
color_dict = {label: color_list[i] for i, label in enumerate(unique_labels)}

# Map colors
label_colors = [color_dict[l] for l in labels]
aligned_cluster_colors = [color_dict[l] for l in aligned_preds]

In [ ]:
#class_names = dataset.class_names  # List of class names in order
#class_name_dict = {i: name for i, name in enumerate(class_names)}
class_name_dict = {
    0: "ARMD",
    1: "Cataract",
    2: "Diabetes",
    3: "Glaucoma",
    4: "Hypertension",
    5: "Myopia",
    6: "Normal",
    7: "Other"
}

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Plot 1: True Labels ---
for label in unique_labels:
    idx = labels == label
    axes[0].scatter(X_2d[idx, 0], X_2d[idx, 1], c=[color_dict[label]],
                    label=class_name_dict[label], s=20)
axes[0].set_title('t-SNE Colored by True Labels')
axes[0].set_xlabel('t-SNE 1')
axes[0].set_ylabel('t-SNE 2')
axes[0].legend(title="True Classes", loc="best", fontsize='small', markerscale=1.5)

# --- Plot 2: Cluster Assignments (Aligned to True Labels) ---
for label in unique_labels:
    idx = aligned_preds == label
    axes[1].scatter(X_2d[idx, 0], X_2d[idx, 1], c=[color_dict[label]],
                    label=class_name_dict[label], s=20)
axes[1].set_title('t-SNE Colored by K-Means Clusters')
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].legend(title="Cluster to Class", loc="best", fontsize='small', markerscale=1.5)

plt.tight_layout()
plt.show()

**Datset 2**

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/My Drive/Ocular Disease Dataset 2/dataset",
    image_size=(224, 224),   # Resize all images
    batch_size=512,         # Get individual images
    shuffle=False            # Keep order for label tracking
)

In [ ]:
# Preprocessing using .map()
dataset = dataset.map(lambda x, y: (preprocess_input(x), y))

# Load pretrained model
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Extract features in batches
features = []
labels = []

for batch_images, batch_labels in dataset:
    batch_features = base_model.predict(batch_images)
    features.append(batch_features)
    labels.append(batch_labels.numpy())

features = np.concatenate(features)
labels = np.concatenate(labels)

In [ ]:
#Right now using K-Means, can try DBSCAN or Agglomerative Clustering
n_clusters = len(np.unique(labels))  # Number of actual classes
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
pred_clusters = kmeans.fit_predict(features)
# Now you have a feature matrix (features) for your images.

In [ ]:
ari = adjusted_rand_score(labels, pred_clusters)
print(f"Adjusted Rand Index: {ari:.4f}")

nmi = normalized_mutual_info_score(labels, pred_clusters)
print(f"NMI: {nmi:.4f}")

In [ ]:
X_2d = TSNE(n_components=2, random_state=42).fit_transform(features)

In [ ]:
# Ensure labels and clusters are numpy arrays
labels = np.array(labels)
pred_clusters = np.array(pred_clusters)

# Step 1: Define a consistent set of classes
all_classes = np.unique(np.concatenate([labels, pred_clusters]))
class_to_index = {cls: i for i, cls in enumerate(sorted(all_classes))}

# Step 2: Choose a colormap with enough colors
cmap = plt.get_cmap('tab10')  # Or use 'tab20' for more classes
num_colors = len(all_classes)
colors = [cmap(i % cmap.N) for i in range(num_colors)]  # wrap around if too many

# Step 3: Build class-to-color mapping
color_dict = {cls: colors[i] for cls, i in class_to_index.items()}

# Step 4: Map each label to a color using the dict
label_colors = [color_dict[l] for l in labels]
cluster_colors = [color_dict[c] for c in pred_clusters]

In [ ]:
def map_clusters_to_labels(true_labels, cluster_preds):
    labels = np.zeros_like(cluster_preds)
    for cluster_id in np.unique(cluster_preds):
        mask = cluster_preds == cluster_id
        labels[mask] = mode(true_labels[mask])[0]
    return labels

aligned_preds = map_clusters_to_labels(labels, pred_clusters)

In [ ]:
# Create color map based on unique true labels
unique_labels = np.unique(labels)
cmap = plt.get_cmap('tab10')
color_list = [cmap(i) for i in range(len(unique_labels))]
color_dict = {label: color_list[i] for i, label in enumerate(unique_labels)}

# Map colors
label_colors = [color_dict[l] for l in labels]
aligned_cluster_colors = [color_dict[l] for l in aligned_preds]

In [ ]:
#class_names = dataset.class_names  # List of class names in order
#class_name_dict = {i: name for i, name in enumerate(class_names)}
class_name_dict = {
    0: "Cataract",
    1: "Diabetes",
    2: "Glaucoma",
    3: "Normal",
}

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Plot 1: True Labels ---
for label in unique_labels:
    idx = labels == label
    axes[0].scatter(X_2d[idx, 0], X_2d[idx, 1], c=[color_dict[label]],
                    label=class_name_dict[label], s=20)
axes[0].set_title('t-SNE Colored by True Labels')
axes[0].set_xlabel('t-SNE 1')
axes[0].set_ylabel('t-SNE 2')
axes[0].legend(title="True Classes", loc="best", fontsize='small', markerscale=1.5)

# --- Plot 2: Cluster Assignments (Aligned to True Labels) ---
for label in unique_labels:
    idx = aligned_preds == label
    axes[1].scatter(X_2d[idx, 0], X_2d[idx, 1], c=[color_dict[label]],
                    label=class_name_dict[label], s=20)
axes[1].set_title('t-SNE Colored by K-Means Clusters')
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].legend(title="Cluster to Class", loc="best", fontsize='small', markerscale=1.5)

plt.tight_layout()
plt.show()